In [649]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import regex

In [650]:
#URL =  "https://www.psychologytoday.com/us/therapists/or/portland/714186?sid=6109d19cf1992&ref=2"

In [651]:
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"}


In [652]:
#page = requests.get(URL, headers=headers)

In [653]:
#soup = BeautifulSoup(page.content, "html.parser")

In [654]:
#zip_code = soup.find(itemprop="postalcode").get_text()

In [655]:
#name = soup.find(itemprop="name").get_text()

In [656]:
#title = soup.find("h2").get_text()

In [657]:
#specialties = soup.find("ul", class_="attribute-list specialties-list").get_text()

In [658]:
#insurance = soup.find("ul", class_= "attribute-list copy-small").get_text()

In [659]:
#price = soup.find("div", class_="finances-office").get_text()#.contents

In [660]:
#price = price.split('\n')

In [661]:
# while '' in price:
#     price.remove('')

In [662]:
records = []
next_url = "https://www.psychologytoday.com/us/therapists/or/portland/108093?sid=610a32c3d201b&ref=1"
for person in range(10):

    next_page = requests.get(next_url, headers=headers)
    next_soup = BeautifulSoup(next_page.content, "html.parser")
    next_zip_code = next_soup.find(itemprop="postalcode").get_text()
    next_name = next_soup.find(itemprop="name").get_text()
    next_title = next_soup.find("h2").get_text()
    try:
        next_specialties = next_soup.find("ul", class_="attribute-list specialties-list").get_text()
    except AttributeError:
        next_specialties = "N/A"
    # next_insurance = next_soup.find("ul", class_= "attribute-list copy-small").get_text()
    try:
        next_price = next_soup.find("div", class_="finances-office").get_text().split('\n')
        while '' in next_price:
            next_price.remove('')
        while '            ' in next_price:
            next_price.remove('            ')
        while '                                                    ' in next_price:
            next_price.remove('                                                    ')
        price = next_price[0]
    except AttributeError:
        price = "N/A"
    
    
    records.append((next_name.strip(), next_zip_code, next_title, next_specialties, price))

    
    time.sleep(1)
    next_url = next_soup.find("a", class_="profile-next")["href"]
#print(records)

In [663]:
df = pd.DataFrame(records, columns=["name","zip","title","specialties","price"])


In [664]:
df["specialties"] = df["specialties"].str.replace(' ','')

In [665]:
df[["title","specialties","price"]] = df[["title","specialties","price"]].replace('\n',' ', regex=True)

In [666]:
df["price"] = df["price"].str.replace('Cost per Session: ','')

In [667]:
df["price"] = df["price"].str.replace(' ','')

In [668]:
df["title"] = df["title"].str.replace(' ','')

In [671]:
#df

In [670]:
df.to_csv('therapist_data.csv')